In [3]:
import arff
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from operator import attrgetter, itemgetter
from io import StringIO
from sklearn.model_selection import train_test_split
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, confusion_matrix,classification_report,precision_recall_fscore_support as score, average_precision_score
from sklearn import metrics
from sklearn.model_selection import cross_validate
import pickle

## training model
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [10]:
data = arff.load(open('../OffComBR3.arff'))
df = pd.DataFrame(data['data'])
df.columns = ['hate', 'sentence']

# transforming 'yes' into 1 and 'no' into 0
df['hate'] = df['hate'].apply(lambda x: 1 if x == 'yes' else 0)

X = df['sentence'].tolist()
y = df['hate'].tolist()
X_train, X_test, y_train, y_test = train_test_split(
                                    X, y, test_size=0.33, random_state=42)

pt_stop_words = nltk.corpus.stopwords.words('portuguese')
classifiers = []
max_df = 0.6

In [11]:
RandomForest =  Pipeline([
        ('tfidf',TfidfVectorizer(ngram_range=(1,4),
                                 lowercase=True,
                                 strip_accents='ascii',
                                 max_df=max_df
                                )),
        ('clf', RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0,)),
        ])
RandomForest.fit(X_train, y_train)
classifiers.append(('randomforest', RandomForest))

In [12]:
MLP = Pipeline([
        ('tfidf',TfidfVectorizer(ngram_range=(1, 1),                                  
                                 lowercase=True,
                                 strip_accents='ascii',
                                 max_df=max_df)),
        ('clf', MLPClassifier(activation='logistic', alpha=0, solver='lbfgs')),
        ])
MLP.fit(X_train, y_train)
classifiers.append(('mlp', MLP))

In [13]:
SVCl = Pipeline([
        ('tfidf',TfidfVectorizer(ngram_range=(1, 1),                        
                                 lowercase=True,
                                 strip_accents='ascii',
                                 max_df=max_df)),
        ('clf', SVC(C=4, kernel='linear', probability=True, shrinking=True, tol=1)),
        ])
SVCl.fit(X_train, y_train)
classifiers.append(('svc', SVCl))

In [15]:
multiNB =  Pipeline([
        ('tfidf',TfidfVectorizer(ngram_range=(1, 1))),
        ('clf', MultinomialNB(alpha=0.1, fit_prior=False)),
        ])
multiNB.fit(X_train, y_train)
classifiers.append(('multinb', multiNB))

In [16]:
SGD = Pipeline([
        ('tfidf',TfidfVectorizer(ngram_range=(1, 1))),
        ('clf', SGDClassifier(alpha=0.01, loss='perceptron', penalty='none')),
        ])
SGD.fit(X_train, y_train)
classifiers.append(('sgd', SGD))

/home/shivwa/.virtualenvs/tcc/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [23]:
DecisionTree = Pipeline([
        ('tfidf',TfidfVectorizer(ngram_range=(1, 1))),
        ('clf', DecisionTreeClassifier(class_weight={0: 1, 1: 2}, criterion='gini', min_samples_split=3)),
        ])
DecisionTree.fit(X_train, y_train)
classifiers.append(('decisiontree', DecisionTree))

In [24]:
for classifier in classifiers:
    pred = classifier[1].predict(X_test)
    print(classifier[0])
    print(classification_report(y_test, pred))

randomforest
              precision    recall  f1-score   support

           0       0.82      0.93      0.87       267
           1       0.51      0.26      0.34        74

   micro avg       0.79      0.79      0.79       341
   macro avg       0.67      0.59      0.61       341
weighted avg       0.75      0.79      0.76       341

mlp
              precision    recall  f1-score   support

           0       0.86      0.89      0.87       267
           1       0.54      0.47      0.50        74

   micro avg       0.80      0.80      0.80       341
   macro avg       0.70      0.68      0.69       341
weighted avg       0.79      0.80      0.79       341

svc
              precision    recall  f1-score   support

           0       0.84      0.94      0.89       267
           1       0.62      0.34      0.44        74

   micro avg       0.81      0.81      0.81       341
   macro avg       0.73      0.64      0.66       341
weighted avg       0.79      0.81      0.79       341